* Read In File
* Base manipulation of file
* Generate Outut for each year

In [ ]:
import pandas as pd

In [ ]:
file_path = "files/"
file_name = "2023-eea_europa_eu-CarsCO2.csv"
df = pd.read_csv(file_path+file_name)
# todo use pd read with dtypes for better performance
df.head(5)

C:\Users\Mr_Di\AppData\Local\Temp\ipykernel_13380\56055625.py:2: DtypeWarning: Columns (3,28,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)


,ID,Country,VFN,Mp,Mh,Man,MMS,Tan,T,Va,...,Erwltp (g/km),De,Vf,Status,year,Date of registration,Fuel consumption,ech,RLFI,Electric range (km)
0,132193881,DE,IP-MQB37SZ_A3_1036-WVW-1,VOLKSWAGEN,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E13*2007/46*1845*26,A1,DXDBX0AC4,...,1.17,NaN,NaN,F,2023,2023-03-14,6.3,NaN,RL-MQ281_6F_20_001-WVW-1,NaN
1,132193882,DE,IP-03_356_0299-ZFA-1,STELLANTIS,STELLANTIS EUROPE,STELLANTIS EUROPE SPA,NaN,E3*2007/46*0373*33,356,HXS12,...,1.35,NaN,NaN,F,2023,2023-01-27,5.2,NaN,RL-03_BU_334_0112-1C4-1,NaN
2,132193883,DE,IP-MQB37SZ_A0_0564-WVW-1,VOLKSWAGEN,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E13*2007/46*1845*27,A1,DLAAX0AE2,...,1.17,NaN,NaN,F,2023,2023-05-15,6.6,NaN,RL-MQ200_6F_18_019-WVW-1,NaN
3,132193884,DE,IP-0000667-WBA-1,BMW,BMW AG,BAYERISCHE MOTOREN WERKE AG,NaN,E1*2007/46*2063*05,FML2E,11DJ,...,NaN,NaN,NaN,F,2023,2023-11-10,NaN,NaN,RL-0100492-WBA-1,227.0
4,132193885,DE,IP-MEB31AZ_A0_1902-WVW-1,VOLKSWAGEN,VOLKSWAGEN,VOLKSWAGEN AG,NaN,E1*2018/858*00004*12,E2,4ACX1EBL1GX1,...,NaN,NaN,NaN,F,2023,2023-08-10,NaN,NaN,RL-EQ151_1K_21_001-WVW-1,491.0


## Processing to create merged file

* remove empty rows
* sort data into common scheme (categorical vars | numerical vars | target)

In [3]:
# pick selection of columns to reduce size of df (less compute)
columns_to_analyse = [
    "ID",
    # "Ct",  # skipped to reduce noise (information included in Cr category_of_vehicle)
    "Cr",
    "m (kg)",
    # "Mt",  # skipped to reduce noise (information included in m (kg))
    "Ewltp (g/km)",  # skipped due to target trap
    "Ft",
    # "Fm",  # skipped to reduce noise (information included in fuel_type)
    "ec (cm3)",
    "ep (KW)",
    # "IT",  # skipped to reduce noise and due to missing information about the variable
    # "Erwltp (g/km)",  # skipped to reduce noise (related to IT)
    "year",
    "z (Wh/km)",
    "Fuel consumption ",
]
col_rename_dict = {
    "ID": "id",
    # "Ct": "category_of_vehicle_type",
    "Cr": "category_of_vehicle",
    "m (kg)": "mass_vehicle",
    # "Mt": "weltp_test_mass",
    "Ewltp (g/km)": "specific_co2_emissions",
    "Ft": "fuel_type",
    # "Fm": "fuel_mode",
    "ec (cm3)": "engine_capacity",
    "ep (KW)": "engine_power",
    # "IT": "innovative_technologies",
    # "Erwltp (g/km)": "erwltp",  # Emissions reduction through innovative technologies (WLTP)
    # "year": "",
    "z (Wh/km)": "electric_energy_consumption",
    "Fuel consumption ": "fuel_consumption",
}
df = df[columns_to_analyse].rename(columns=col_rename_dict)

Change order of variables: categorical, numerical, target

In [4]:
# Sort df (categorical vars => numerical vars => target)
cat_vars = df.select_dtypes(include="object")
num_vars = df.select_dtypes(include=["float64", "int64"])

In [5]:
# choose potential targets: 'electric_energy_consumption', 'fuel_consumption', 'specific_co2_emissions'
potential_targets = ['electric_energy_consumption', 'fuel_consumption', 'specific_co2_emissions']
col_order = list(cat_vars.columns)
col_order.extend(num_vars.columns)

# targets at last
for target in potential_targets:
    if target in col_order:
        col_order.remove(target)
    col_order.append(target)

print(col_order)

['category_of_vehicle', 'fuel_type', 'id', 'mass_vehicle', 'engine_capacity', 'engine_power', 'year', 'electric_energy_consumption', 'fuel_consumption', 'specific_co2_emissions']


In [6]:
# overwrite df with sorted col_order
df = df.loc[:, col_order]

cat_vars = df.select_dtypes(include="object")
num_vars = df.select_dtypes(include=["float64", "int64"])

In [ ]:
# Output

df.to_csv(file_path+"output/merge_"+file_name)